In [ ]:
# Imports iniciais
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output
import itertools

# === CARREGAR MODELO TREINADO ===
modelo = joblib.load('modelo_4_camadas_ESRS_ESRD_COMPLETO.pkl')
weights = modelo['weights']
scaler = modelo['scaler']
aeronaves_dict = modelo['aeronaves']

# === FUNÇÕES RNA ===
def tanh(x): return np.tanh(x)
def neural_network(weights, X):
    W1 = weights[:9 * 10].reshape((9, 10))
    b1 = weights[90:100]
    W2 = weights[100:180].reshape((10, 8))
    b2 = weights[180:188]
    W3 = weights[-9:-1].reshape((8, 1))
    b3 = weights[-1]
    hidden1 = tanh(np.dot(X, W1) + b1)
    hidden2 = tanh(np.dot(hidden1, W2) + b2)
    output = np.dot(hidden2, W3) + b3
    return output.flatten()

def prever_deformacao_customizada(valores):
    entrada = np.array([valores])
    entrada_norm = scaler.transform(entrada)
    return neural_network(weights, entrada_norm)[0]

# === AUSTROADS: Temperatura média anual ===
def calcular_temperaturas(temp_media_anual):
    return None, temp_media_anual

def corrigir_mr_para_Tc(E25_mpa, temp_media_anual):
    return E25_mpa * np.exp(-0.08 * (temp_media_anual - 25))

# === INTERFACE PARA CONFIABILIDADE ===
covs_padrao = {
    'revest_MR': 15, 'base_MR': 20, 'subbase_MR': 20, 'subleito_MR': 20,
    'revest_h': 7, 'base_h': 12, 'subbase_h': 15
}

cov_widgets = {
    key: widgets.FloatText(value=val, description=f'COV {key} (%)')
    for key, val in covs_padrao.items()
}

nivel_confianca = widgets.FloatText(value=95.0, description='Confiabilidade (%)')

confiab_section = widgets.VBox([
    widgets.HTML('<b>Coeficientes de Variação (COV):</b>')
] + list(cov_widgets.values()) + [nivel_confianca])

display(confiab_section)
